### Autograd: Engine in pytorch to automatically compute gradients for backpropogation

Simple one layer NN

Gradients are required for w and b, so we set the requires_grad=True for those tensors

In [12]:
import torch

#input tensor
x = torch.ones(5)

#output
y = torch.zeros(3)

w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
loss

tensor(0.5181, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

Computation Graph for this operation is

Reference to grad object of z

In [5]:
z.grad_fn


In [6]:
loss.grad_fn

Partial derivatives of loss, wrt w and b

∂loss/∂w and ∂loss/∂b under some fixed values of x and y

In [7]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1811, 0.1727, 0.0131],
        [0.1811, 0.1727, 0.0131],
        [0.1811, 0.1727, 0.0131],
        [0.1811, 0.1727, 0.0131],
        [0.1811, 0.1727, 0.0131]])
tensor([0.1811, 0.1727, 0.0131])


##### We can get grad properties for only leaf nodes of the computational graph

Disable Gradient Tracking

In [13]:
with torch.no_grad():
    z = torch.matmul(x, w)+b
z.requires_grad

False

In [15]:
z = torch.matmul(x, w)+b
z_det = z.detach()
z_det.requires_grad

False

- This can be used to freeze cetain params while finetuning a pretrained model
- Also, when only forward pass is req

- Autograd keeps a record of tensor and all executed operations in a DAG
- Leaves are input tensors and root is output tensor
- Tracing from root to leaves, gradients can be computed using chain rule